In [20]:
%matplotlib ipympl
#%matplotlib inline

For high dpi displays.

In [21]:
%config InlineBackend.figure_format = 'retina'

# 0. Note

This notebook demonstrates how different physical parameters change the equation of state.

# 1. Setup

In [22]:
import matplotlib.pyplot as plt
import numpy as np
from uncertainties import unumpy as unp
import pytheos as eos
from uncertainties import ufloat

In the Mie-Gruneisen approach, we divide total pressure, $P(V,T)$, into static pressure, $P(V,T_0)$, which is a reference isotherm (normally $T_0 = 300$ K), and thermal pressure, $\Delta P_\mathrm{th}(V,T)$, which is a pressure difference between 300-K and high-temperature isotherms.

\begin{equation}
P(V,T) = P(V,T_0) + \Delta P_\mathrm{th}(V,T).\label{eq:eos-totalP}
\end{equation}

# 2. Static pressure

We will use the Birch-Murnathan equation for the static part.

\begin{equation}
P = \frac{3K_0}{2} \left[ \left(\frac{V_0}{V}\right)^{7/3} -
\left(\frac{V_0}{V}\right)^{5/3} \right]\left\{
1-\xi\left[\left(\frac{V_0}{V}\right)^{2/3} -1\right] \right\},
\end{equation}
where $K_0$ is the bulk modulus (in GPa), $K'_0 = (dK/dP)$, and $\xi = \frac{3}{4}(4-K_0')$.


Generate volume strain: $V/V_0$.

In [23]:
eta = np.linspace(1., 0.75, 21)
print(eta)

[1.     0.9875 0.975  0.9625 0.95   0.9375 0.925  0.9125 0.9    0.8875
 0.875  0.8625 0.85   0.8375 0.825  0.8125 0.8    0.7875 0.775  0.7625
 0.75  ]


Suppose you have a material with the following properties measured at 300 K.

In [24]:
v0 = 163. # A^3
k0 = ufloat(260., 3.) # GPa
k0p = ufloat(4., 0.2)  

In [25]:
v = unp.uarray(eta * v0, 0.05)

Then you can calculate pressures for the volumes at 300 K: a 300-K isotherm.

In [26]:
p_st = eos.bm3_p(v, v0, k0, k0p)
print(p_st)

[0.0+/-0.07975460122699388 3.353814280180503+/-0.09340510990896281
 6.924610875374034+/-0.12194338729592742
 10.727242151995076+/-0.16222403233727498
 14.777735537752497+/-0.21340786649401286
 19.093401407355305+/-0.2762953014513782
 23.69295235774493+/-0.3525063404100263
 28.596635240437596+/-0.4440895414919564
 33.82637750346423+/-0.5533700034339936
 39.40594960818801+/-0.6829058519759348
 45.361145531635096+/-0.8354943541892824
 51.719983648420246+/-1.0142018197089469
 58.51293061441608+/-1.2224059635435147
 65.77315125481373+/-1.463846093588458
 73.53678790143398+/-1.7426796902726758
 81.84327313916256+/-2.063545519649513
 90.73568052253206+/-2.431634226752447
 100.26111852669689+/-2.8527678157175513
 110.47117382163177+/-3.333489748417916
 121.4224109275685+/-3.8811676912523003
 133.17693645161637+/-4.504111266618983]


In [27]:
plt.errorbar(unp.nominal_values(p_st), unp.nominal_values(v), \
            xerr=unp.std_devs(p_st), yerr=unp.std_devs(v)) #, xerr=unp.std_devs(p_st))
plt.xlabel('Pressure')
plt.ylabel('Volume')

FigureCanvasNbAgg()

Text(0,0.5,'Volume')

## Try these...

- Increase bulk modulus, $K_0$, to 300 GPa.

- Increase pressure derivative of bulk modulus, $K'_0$, to 3 and 5.

What did you find?

Can you convert this into a density profile along an isotherm?

# 3. Thermal pressure

Now let's calculate thermal pressure.  We will use the Debye approach.

\begin{equation}
\Delta P_\mathrm{th}(V,T) = \frac{\gamma(V)}{V}\Delta E_\mathrm{th}[\theta_D(V),T],\label{eq:eos-MGD}
\end{equation}
where $\gamma$ is the Gruneisen parameter ($= V (\partial P/\partial E)_V$), and $T_0=300$ K in most experiments.  The internal energy change can be calculated from the Debye model:

\begin{equation}
E_\mathrm{th} = \frac{9nRT}{x^3}\int^x _0 \frac{\xi^3}{e^\xi -1} d\xi,
\end{equation}
where $R$ is the gas constant, $x = \dfrac{\theta_D}{T}$, and $n$ is the number of atoms per formula unit.

Also,

\begin{equation}
\gamma = \gamma_0 \left( \dfrac{V}{V_0} \right) ^q
\end{equation}



We need some parameters for the Debye thermal pressure equation.

In [28]:
temp = 2000.
gamma0 = ufloat(1.5, 0.3)
q = ufloat(1., 0.5)
theta0 = ufloat(1000., 50.)

In [29]:
n = 5.
z = 4.

In [30]:
p_th = eos.constq_pth(v, temp, v0, gamma0, q, theta0, n, z)

In [31]:
p_ht = p_st + p_th
print(p_ht)

[12.018782150648697+/-2.4064669154975995
 15.341232585875414+/-2.3956040507971004
 18.879762054847024+/-2.387408699537276
 22.649205498278228+/-2.3822528435555035
 26.66557306440265+/-2.380700743759324
 30.946158023609556+/-2.383576342095162
 35.50965607313762+/-2.3920446642608924
 40.37629739931588+/-2.407706376702418
 45.56799304974008+/-2.4327017308806176
 51.10849738055938+/-2.469815544534059
 57.023588589399225+/-2.522569265645685
 63.34126962788805+/-2.5952809047949947
 70.09199211581935+/-2.6930720116057394
 77.30890625947735+/-2.8218074097037507
 85.02814021886469+/-2.987970085762949
 93.28911288358483+/-3.1984978721774473
 102.13488461828229+/-3.460628760473617
 111.61255124176922+/-3.7818064334551034
 121.7736873285561+/-4.1696812047459835
 132.67484589069147+/-4.632212942996751
 144.37812263975508+/-5.1778574217801765]


Let's plot 300-K and 3000-K isotherms together.

In [32]:
plt.errorbar(unp.nominal_values(p_st), unp.nominal_values(v), xerr=unp.std_devs(p_st))
plt.errorbar(unp.nominal_values(p_ht), unp.nominal_values(v), xerr=unp.std_devs(p_ht))
plt.xlabel('Pressure')
plt.ylabel('Volume')

Text(92.7917,0.5,'Volume')

## Try these...

- Increase Gruneisen parameter, $\gamma_0$, to 3.

- Increase logarithmic volume dependence of $\gamma$, $q$, to 0.1 and 2.

What did you find?

Can you convert this into a density profile along an isotherm?

# 4. Volume expansion

In [14]:
import pytheos.scales.objs
from collections import OrderedDict

In [15]:
params_st = OrderedDict([('v0', ufloat(v0, 0.01)),
                         ('k0', k0),
                         ('k0p', k0p)])
params_th = OrderedDict([('v0', ufloat(v0, 0.01)),
                         ('gamma0', gamma0),
                         ('q', q),
                         ('theta0', theta0)])
brd = pytheos.scales.objs.MGEOS(n, z, params_st=params_st, params_th=params_th,
               eqn_st='bm3', eqn_th='constq')

In [16]:
p_lm = np.linspace(25., 125., 101)

In [17]:
v_r = brd.cal_v(p_lm, np.ones_like(p_lm)*1500.)
v_ht = brd.cal_v(p_lm, np.ones_like(p_lm)*2000.)

In [18]:
plt.plot(p_lm, (v_ht - v_r) / v_r * 100.)
plt.ylabel('$\Delta V / V$ (%)')
plt.xlabel('Pressure (GPa)')

Text(0.5,0,'Pressure (GPa)')